In [74]:
import pandas as pd
import psycopg2
from geopy.distance import geodesic


In [75]:

coordinates = pd.read_csv('corrected_latitude_longitude_details.csv')
terrain = pd.read_csv('terrain_classification.csv')


In [76]:
coordinates['km'] = km_distances
terrain['km']=terrain['distance (in km)']
merged_data = pd.merge_asof(coordinates.sort_values('km'), terrain.sort_values('km'), on='km')


In [77]:
conn = psycopg2.connect("dbname=ranjitaelizabeth user=ranjitaelizabeth password= host=localhost port=5432")
cur = conn.cursor()


In [78]:
cur.execute("""
CREATE TABLE IF NOT EXISTS route (
    id SERIAL PRIMARY KEY,
    latitude DOUBLE PRECISION,
    longitude DOUBLE PRECISION,
    km DOUBLE PRECISION,
    terrain VARCHAR(255)
)
""")


In [79]:
for index, row in merged_data.iterrows():
    cur.execute("""
    INSERT INTO route (latitude, longitude, km, terrain)
    VALUES (%s, %s, %s, %s)
    """, (row['latitude'], row['longitude'], row['km'], row['terrain']))


In [80]:
conn.commit()
cur.close()
conn.close()

In [81]:
conn = psycopg2.connect("dbname=ranjitaelizabeth user=ranjitaelizabeth password= host=localhost port=5432")
cur = conn.cursor()


In [82]:
cur.execute("""
SELECT latitude, longitude, km, terrain 
FROM route 
WHERE terrain LIKE '%road%' AND terrain NOT LIKE '%civil station%'
""")
results = cur.fetchall()


In [83]:
df = pd.DataFrame(results, columns=['latitude', 'longitude', 'km', 'terrain'])
df.to_csv('road_without_civil_station.csv', index=False)
df


,latitude,longitude,km,terrain
0,10.004526,76.375986,0.000000,"boundary wall,road"
1,10.004385,76.375502,0.055318,"boundary wall,road"
2,10.004391,76.375115,0.097699,"boundary wall,road"
3,10.004453,76.374769,0.136255,"boundary wall,road"
4,10.004532,76.374565,0.160288,"boundary wall,road"
...,...,...,...,...
151,10.006775,76.372777,1.032689,road
152,10.006695,76.373054,1.064411,road
153,10.006753,76.373492,1.112800,road
154,10.006858,76.373783,1.146794,road


In [84]:
cur.close()
conn.close()

In [85]:
conn = psycopg2.connect("dbname=ranjitaelizabeth user=ranjitaelizabeth password= host=localhost port=5432")
cur = conn.cursor()

In [86]:
def extract_boundary_points(km_value):
    cur.execute("""
    SELECT latitude, longitude, km
    FROM route
    WHERE km >= %s
    ORDER BY km ASC
    LIMIT 1
    """, (km_value,))
    start_point = cur.fetchone()

    cur.execute("""
    SELECT latitude, longitude, km
    FROM route
    WHERE km <= %s
    ORDER BY km DESC
    LIMIT 1
    """, (km_value,))
    end_point = cur.fetchone()

    return start_point, end_point


In [87]:
km_value = 10  # Example KM value
start, end = extract_boundary_points(km_value)
print(f"Start Point: {start}")
print(f"End Point: {end}")


Start Point: None
End Point: (10.006894, 76.373876, 1.1576991689665062)


In [88]:
cur.close()
conn.close()